# Compare Old and New STRait Razor Results

We first tried using STRait Razor with .config files that were generated directly from PERF that was run on the reference .fasta sequence for all thirty primers. Each .config file corresponded to a single primer and contained one 5' and 3' flanking region pair for the STR. 

We found that running STRait Razor resulted in incorrect genotypes when indels were present (Primer 25) - this was due to the .config file not containing flanking region options for samples with and without the indel. We also found that results were only available for 159 out of 196 possible genotypes, but for these 159 primer/sample pairs, STRait Razor had 75% accuracy.

To make .config files that are robust to indels, we ran PERF on every single read for all primer/sample pairings, and then chose the most frequent flanking regions for our .config files (a flanking region was deemed "frequent" if it contained > 7.5% of all reads, forward and reverse, since we chose 15% as our allele identification threshold earlier). (***this threshold needs to be fixed***). We reasoned that including all reads would generate .config files that contained not just one haplotype, but many haplotypes for the flanking regions, allowing all samples to be "matched" to flanking regions and then genotyped by STRait Razor.

Our .config files on average tended to have more than one line - many lines were repeats with only one base pair off, but others contained the repeat motif and were "false" flanking regions. However, many lines did account for indels. Nonetheless, these .config files need to be fixed with a "sliding window" approach to process false flanking regions, but before doing so, we need to identify which primers and samples were the most problematic for STRait razor run using the new .config files, which had ~60% accuracy, but for 172 primer/sample pairs.


In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

## See which samples failed before/after new .config file creation
"Failed" in this case means that the STRait razor output file was empty, not that the genotyping was incorrect

In [2]:
#Load in all excel table genotypes
excel_pyrhulla_genotypes = pd.read_csv("C:/Users/vik16/OneDrive/Documents/FOGS Summer 2022/" +\
                                       "excel_pyrhulla_genotypes.tsv",sep="\t")

#Load in all STRait razor genotypes using multiple flanks (new config files) or single flanks (old config files)
full_merged_multiple_flanks_tsv = pd.read_csv("C:/Users/vik16/OneDrive/Documents/FOGS Summer 2022/STRaitRazor/results/" +\
                             "alleles/full_merged_STRait_excel_alleles_0.15_multiple_flanks.tsv",sep="\t")
pd.options.display.max_colwidth = 100
full_merged_multiple_flanks_tsv = full_merged_multiple_flanks_tsv.reindex(sorted \
                                        (full_merged_multiple_flanks_tsv.columns), axis=1)
full_merged_single_flanks_tsv = pd.read_csv("C:/Users/vik16/OneDrive/Documents/FOGS Summer 2022/STRaitRazor/results/" +\
                             "alleles/full_merged_STRait_excel_alleles_0.15_flank10.tsv",sep="\t")
full_merged_single_flanks_tsv = full_merged_single_flanks_tsv.reindex(sorted \
                                        (full_merged_single_flanks_tsv.columns), axis=1)

#See if any primers are discarded by STRait Razor in either analysis
print("Primers in Excel Table:")
print("**********************************************")
print(np.unique(excel_pyrhulla_genotypes[["Primer"]]))

print("Primers genotyped using old .config files:")
print("**********************************************")
print(np.unique(full_merged_single_flanks_tsv[["Primer"]]))
    
print("Primers genotyped using new .config files:")
print("**********************************************")
print(np.unique(full_merged_multiple_flanks_tsv[["Primer"]]))
print("")
#See which primer/sample pairs from the excel table are discarded by STRait Razor in either analysis

excel_pyrhulla_genotypes["P/S Pair"]=excel_pyrhulla_genotypes[["Primer","Sample"]].apply(tuple,axis=1)
full_merged_multiple_flanks_tsv["P/S Pair"]=full_merged_multiple_flanks_tsv[["Primer","Sample"]].apply(tuple,axis=1)
full_merged_single_flanks_tsv["P/S Pair"]=full_merged_single_flanks_tsv[["Primer","Sample"]].apply(tuple,axis=1)

print("Failed pairings after new .config files:")
print("*****************************************************")
multiple_flanks_failed_pairings=excel_pyrhulla_genotypes["P/S Pair"][~excel_pyrhulla_genotypes["P/S Pair"] \
                   .isin(full_merged_multiple_flanks_tsv["P/S Pair"])]
print(multiple_flanks_failed_pairings)
print("")

print("Failed pairings after old .config files:")
print("*****************************************************")
single_flanks_failed_pairings=excel_pyrhulla_genotypes["P/S Pair"][~excel_pyrhulla_genotypes["P/S Pair"] \
                   .isin(full_merged_single_flanks_tsv["P/S Pair"])]
print(single_flanks_failed_pairings)
print(len(single_flanks_failed_pairings))
print("")

print("Pairings that originally failed prior to new .config files, but worked after:")
new_config_new_genotypes=single_flanks_failed_pairings[~single_flanks_failed_pairings.isin(multiple_flanks_failed_pairings)]
print(new_config_new_genotypes)

Primers in Excel Table:
**********************************************
[ 3  4  5  8 10 11 12 13 14 15 16 17 18 19 20 22 23 25 26 28 29 30]
Primers genotyped using old .config files:
**********************************************
[ 3  4  8 10 11 12 13 14 15 16 17 18 20 22 23 25 26 28 29 30]
Primers genotyped using new .config files:
**********************************************
[ 3  4  5  8 10 11 12 13 14 15 16 17 18 20 22 23 25 26 28 29 30]

Failed pairings after new .config files:
*****************************************************
20         (5, 1232)
21         (5, 1393)
29        (5, 34966)
81     (14, 2006174)
85        (15, 1393)
89         (15, 217)
91     (15, 2599208)
110       (18, 1232)
117       (19, 1232)
118       (19, 1393)
119    (19, 2006092)
120        (19, 217)
121    (19, 2520669)
122    (19, 2599208)
123      (19, 34896)
124      (19, 34966)
167    (26, 2006092)
168        (26, 217)
184    (29, 2006174)
186      (29, 34966)
191    (30, 2006174)
192        (30, 2

## Incorrect Genotyping Comparison
* See which pairs "failed" (empty results) with old .config files but were wrongly genotyped with new .config files
* See which pairs "failed" (empty results) with old .config files but were correctly genotyped with new .config files
* See which pairs were correctly genotyped with old .config files but not with new .config files
* See which pairs were correctly genotyped with new .config files but not with old .config files
*etc.

In [4]:
#Load in and combine mismatch data
full_mismatch_multiple_flanks_tsv = pd.read_csv("C:/Users/vik16/OneDrive/Documents/FOGS Summer 2022/STRaitRazor/results/" +\
                             "alleles/full_mismatch_STRait_razor_alleles_0.15_multiple_flanks.tsv",sep="\t")
pd.options.display.max_colwidth = 100
full_mismatch_multiple_flanks_tsv = full_mismatch_multiple_flanks_tsv.reindex(sorted \
                                        (full_mismatch_multiple_flanks_tsv.columns), axis=1)
single_mismatch_multiple_flanks_tsv = pd.read_csv("C:/Users/vik16/OneDrive/Documents/FOGS Summer 2022/STRaitRazor/results/" +\
                             "alleles/single_mismatch_STRait_razor_alleles_0.15_multiple_flanks.tsv",sep="\t")
pd.options.display.max_colwidth = 100
single_mismatch_multiple_flanks_tsv = single_mismatch_multiple_flanks_tsv.reindex(sorted \
                                        (single_mismatch_multiple_flanks_tsv.columns), axis=1)

all_mismatch_multiple_flanks_tsv = pd.concat((full_mismatch_multiple_flanks_tsv,single_mismatch_multiple_flanks_tsv),
                                            ignore_index=True)
all_mismatch_multiple_flanks_tsv["P/S Pair"] = all_mismatch_multiple_flanks_tsv[["Primer","Sample"]].apply(tuple,axis=1)


full_mismatch_single_flanks_tsv = pd.read_csv("C:/Users/vik16/OneDrive/Documents/FOGS Summer 2022/STRaitRazor/results/" +\
                             "alleles/full_mismatch_STRait_razor_alleles_0.15_flank10.tsv",sep="\t")
pd.options.display.max_colwidth = 100
full_mismatch_single_flanks_tsv = full_mismatch_single_flanks_tsv.reindex(sorted \
                                        (full_mismatch_single_flanks_tsv.columns), axis=1)
single_mismatch_single_flanks_tsv = pd.read_csv("C:/Users/vik16/OneDrive/Documents/FOGS Summer 2022/STRaitRazor/results/" +\
                             "alleles/single_mismatch_STRait_razor_alleles_0.15_flank10.tsv",sep="\t")
pd.options.display.max_colwidth = 100
single_mismatch_single_flanks_tsv = single_mismatch_single_flanks_tsv.reindex(sorted \
                                        (single_mismatch_single_flanks_tsv.columns), axis=1)

all_mismatch_single_flanks_tsv = pd.concat((full_mismatch_single_flanks_tsv,single_mismatch_single_flanks_tsv),
                                            ignore_index=True)
all_mismatch_single_flanks_tsv["P/S Pair"] = all_mismatch_single_flanks_tsv[["Primer","Sample"]].apply(tuple,axis=1)


new_config_errors=all_mismatch_multiple_flanks_tsv["P/S Pair"]
old_config_errors=all_mismatch_single_flanks_tsv["P/S Pair"]


print("Number of pairs that didn't run with old .config files but ran with new .config files:")
print("*****************************************************************")
print(len(new_config_new_genotypes))
print("")


print("Pairs that didn't run with old .config files and were incorrectly genotyped with new .config files:")
print("*****************************************************************")
new_config_not_run_before_errors=new_config_errors[new_config_errors.isin(single_flanks_failed_pairings)]
print(new_config_not_run_before_errors)
print("Length:")
print(len(new_config_not_run_before_errors))

print("Pairs that didn't run with old .config files and were correctly genotyped with new .config files:")
print("*****************************************************************")
new_config_not_run_before_errors=new_config_new_genotypes[~new_config_new_genotypes.isin(new_config_errors)]
print(new_config_not_run_before_errors)
print("Length:")
print(len(new_config_not_run_before_errors))


print("Pairs that were correctly genotyped in old .config files but not new .config files:")
print("*****************************************************************")
new_config_actual_errors=new_config_errors[~new_config_errors.isin(old_config_errors)]
new_config_actual_errors=new_config_actual_errors[~new_config_actual_errors.isin(single_flanks_failed_pairings)]
print(new_config_actual_errors.iloc[np.argsort(new_config_actual_errors)])
print("Length:")
print(len(new_config_actual_errors))


print("Pairs that were correctly genotyped in new .config files but not in old .config files:")
print("*****************************************************************")
old_but_not_new_errors=old_config_errors[~old_config_errors.isin(new_config_errors)]
print(old_but_not_new_errors.iloc[np.argsort(old_but_not_new_errors)])
print("Length:")
print(len(old_but_not_new_errors))
print("")

print("Pairs that were incorrectly genotyped in both analyses:")
print("*****************************************************************")
shared_errors=old_but_not_new_errors=old_config_errors[old_config_errors.isin(new_config_errors)]
print(shared_errors.iloc[np.argsort(shared_errors)])
print("Length:")
print(len(shared_errors))
print("")

Number of pairs that didn't run with old .config files but ran with new .config files:
*****************************************************************
13

Pairs that didn't run with old .config files and were incorrectly genotyped with new .config files:
*****************************************************************
1         (5, 217)
6        (5, 1791)
9       (5, 34896)
13    (5, 2006092)
16    (5, 2006174)
19    (5, 2520669)
20    (8, 2520669)
23    (5, 2599208)
Name: P/S Pair, dtype: object
Length:
8
Pairs that didn't run with old .config files and were correctly genotyped with new .config files:
*****************************************************************
30       (8, 1232)
33    (8, 2006092)
34    (8, 2006174)
38      (8, 34896)
39      (8, 34966)
Name: P/S Pair, dtype: object
Length:
5
Pairs that were correctly genotyped in old .config files but not new .config files:
*****************************************************************
3        (20, 1232)
34       (20, 13